In [1]:
import numpy as np

In [2]:
import requests
import os

In [3]:
import cv2
from PIL import Image

In [4]:
os.chdir('Documents/Projects/ImageRec')

In [5]:
import matplotlib.pyplot as plt

In [6]:
def img_save(query_term, offset):
    '''
    saves and processes images for specified query, creates folder in directory if there isnt one
    '''
    
    API_KEY = os.getenv('bing_search_api_key')
    URL = "https://api.cognitive.microsoft.com/bing/v5.0/images/search"
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)' + \
                 'Chrome/80.0.3987.87 Safari/537.36'
    
    # set path with query term as file name
    path = os.getcwd().replace('\\', '/') + '/' + str(query_term) + '/'
    
    if not os.path.exists(query_term):
        # if the directory does not exist, make one
        os.mkdir(query_term)
        
    headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
    params = {"q": query_term,
              "count": 150, 
              "offset": offset * 150}

    # Search Bing for images
    search = requests.get(URL, headers=headers, params=params)
    results = search.json()
    
    print('TOTAL ESTIMATED MATCHES: ' + str(results['totalEstimatedMatches']))

    # Save all of the resulting images from each page
    num = offset * 150
    for value in results['value']:

        print(str(num) + ' ' + value['contentUrl'])
        
        try:
            image = requests.get(value["contentUrl"], timeout=30, headers={'User-Agent': USER_AGENT})
            
        except(requests.ConnectionError):
            print(str(num) + ' BAD CONNECTION')
        
        # Check the status of the request - If the image does not exist we will skip it
        try:
            image.raise_for_status()
            
            file = open(path + query_term + '_' + str(offset) + '_' + str(num) + '.png', 'wb')
            file.write(image.content)
            file.close()
            
        except(requests.HTTPError):
            print(str(num) + ' NOT FOUND')
            
        num += 1

In [7]:
def img_format(folder):
    '''
    read in all images in given folder, format them, and put them into a master array
    '''
    
    x_data = np.array([])
    Y_data = np.array([])
    
    # Iterate through each file in the specified folder
    for file in os.listdir(folder):
        
        # Read in the image
        img = np.array(Image.open(folder + '/' + file))
        
        # If the image is greyscale, discard it
        if len(img.shape) == 2:
            continue
        
        # If the image is 4 channel (RGBA), convert to 3 channel (RGB)
        if (len(img.shape) > 2) & (img.shape[2] == 4):
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        
        # Identify which sides need to be padded and by how much, to make the image square
        short = np.argmin(img.shape[:2])
        diff_1 = int(np.ceil(abs(img.shape[1] - img.shape[0])/2))
        diff_2 = int(np.floor(abs(img.shape[1] - img.shape[0])/2))
        
        # Set the desired padding on the short side, and apply
        width = [[0, 0], [0, 0], [0, 0]]
        width[short] = [diff_1, diff_2]
        img = np.pad(img, pad_width=width)
        
        # Resize square image to 100x100
        img = cv2.resize(img, (100, 100))
        
        # Reshape array to be appended to x_data array
        img = img.reshape(1, 100, 100, 3)
        
        # Put the formatted arrays into a master array of training data
        if np.array_equal(x_data, np.array([])):
            # This is the first one, start the array
            x_data = img
            
        else:
            # Append to full array
            x_data = np.concatenate((x_data, img), axis=0)
        
        # The class label will be same as the name of the folder
        Y_data = np.append(Y_data, folder)
        
        # fix type
        #x_data = x_data.astype(np.int32)
    
    return x_data, Y_data

Save the images from search queries

In [357]:
img_save('dog', 1)

TOTAL ESTIMATED MATCHES: 489
150 https://www.urdogs.com/wp-content/uploads/2016/06/Husky.jpg
151 http://68.media.tumblr.com/6233ff327f7fefe564f2eaddc76060fe/tumblr_omprbsaWwm1s4yg05o1_1280.jpg
152 https://vcahospitals.com/-/media/vca/images/lifelearn-images/dog-behavior-normal.ashx?la=en&hash=F8B875D7EDFBC5B31AA81A47B34A9100
153 https://i.ytimg.com/vi/2Sxu-cqflsI/maxresdefault.jpg
154 http://diamondwaterscaravanpark.com.au/wp-content/uploads/2016/07/pet-friendly-accommodation-mid-north-coast.jpg
155 http://thebark.com/sites/default/files/styles/full/public/content/blog/full/81c39033-68b8-4096-8f29-a73ef7df2ac5.jpg?itok=jrtNNT6o
156 http://www.publicdomainpictures.net/pictures/40000/nahled/golden-retriever-dog-1362597915Did.jpg
157 http://2.bp.blogspot.com/-U1ftQjlMcdg/UO2gSGvxp8I/AAAAAAAABEE/Gf2SQE2JYwE/s640/Akbash+dog+6.jpg
158 https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Jumping_dog.JPG/220px-Jumping_dog.JPG
159 https://upload.wikimedia.org/wikipedia/commons/6/68/Rob_Roy

256 http://www.dogvills.com/wp-content/uploads/2016/08/Best-Hypoallergenic-Dog-Food-for-Goldens-f.jpg
257 https://specializedpetservices.com/wp-content/gallery/photos/IMG_3083.jpg
258 https://upload.wikimedia.org/wikipedia/commons/2/2d/Pdog.jpg
259 http://www.dog-learn.com/dog-breeds/polish-tatra-sheepdog/images/polish-tatra-sheepdog-u5.jpg
260 http://www.petpaw.com.au/wp-content/uploads/2014/09/Smart-Looking-Basque-Shepherd-Dog-1030x1030.jpg
261 https://s-i.huffpost.com/gen/1115786/images/o-RUNNING-OF-THE-CHIHUAHUAS-facebook.jpg
262 http://4.bp.blogspot.com/-JtelMGttqgs/UIPOdjwaKdI/AAAAAAAAABE/ncCgaU49H9U/w1200-h630-p-k-no-nu/rottweiler_id73.jpg
263 https://s-i.huffpost.com/gen/2487988/images/o-DOG-GOES-TO-BARBER-facebook.jpg
264 https://pbs.twimg.com/profile_images/687509830/4345733231_2b8f1ede14_o.jpg
265 https://upload.wikimedia.org/wikipedia/en/9/9f/Flying_Dog_cover.jpg
266 http://2.bp.blogspot.com/-QHLT6NhwUZM/UHR-YXa3ohI/AAAAAAAABMA/NkSuJa45I7A/s1600/So+Cute+and+Funny+Bulldogs+(

In [358]:
img_save('dog', 2)

TOTAL ESTIMATED MATCHES: 489
300 https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Aksaray_malaklisi_beto.jpg/220px-Aksaray_malaklisi_beto.jpg
301 http://ww1.prweb.com/prfiles/2014/04/30/11813091/FC043.jpg
302 http://cdn.skim.gs/image/upload/v1456338021/msi/dog_pound_q55tos.jpg
303 http://4.bp.blogspot.com/-YqGnAv0gNMI/UP5eHntcTmI/AAAAAAAAARA/z7VhPx0k7rk/s320/mutant+dog.jpg
304 https://static.designboom.com/wp-content/uploads/2017/11/aibo-robot-dog-designboom-600.gif
305 https://ae01.alicdn.com/kf/HTB1EBSKLFXXXXa1XVXXq6xXFXXXv/Wholesale-Cheap-font-b-Dog-b-font-Jumpsuits-Clothes-For-font-b-Dog-b-font-Chihuahua.jpg
306 http://wiki.godvillegame.com/images/thumb/d/df/Toothless_Sun_Dog.jpg/200px-Toothless_Sun_Dog.jpg
307 https://i.pinimg.com/736x/6d/7a/9d/6d7a9d946b04e36437b3f6f0a982fb9d--belgian-malinois-puppies-malinois-dog.jpg
308 http://starschanges.com/wp-content/uploads/2016/08/sandra-bullock-dog-poppy.jpg
309 http://3.bp.blogspot.com/-mFP84hcSGqs/T2FYbp-QbCI/AAAAAAAAArc/I1IvX

377 https://upload.wikimedia.org/wikipedia/commons/a/ab/Brown_Dog_statue%2C_Battersea%2C_London%282%29.jpg
378 https://upload.wikimedia.org/wikipedia/en/2/2e/Cold_Dog_Soup_%281990_film%29.jpg
379 https://upload.wikimedia.org/wikipedia/en/c/c6/Dog_Days_preview_poster.jpg
380 https://lmiller82691.files.wordpress.com/2015/02/mcgruffthecrimedog.jpg?w=1022
381 https://i.pinimg.com/originals/c6/b2/99/c6b29945b786f98c6e9e6eec5406fd33.jpg
382 https://upload.wikimedia.org/wikipedia/en/thumb/5/52/Lenny_the_Wonder_Dog_DVD_cover.jpg/220px-Lenny_the_Wonder_Dog_DVD_cover.jpg
383 https://upload.wikimedia.org/wikipedia/en/8/89/DOGTV_Logo.png
384 https://upload.wikimedia.org/wikipedia/en/6/65/SweetNothingsDogFashionDisco.jpg
385 https://vignette2.wikia.nocookie.net/muppet/images/b/b8/DimplestheDog.jpg/revision/latest/scale-to-width-down/259?cb=20141015204636
386 https://i.pinimg.com/736x/db/d4/b3/dbd4b3322af416f147790b920ba5a970--dog-shaming-photos-funny-dog-shaming.jpg
387 https://upload.wikimedia.org

448 https://www.dogstrust.org.uk/dogimages/1082961_pippa_20200206113749_pippa-1_800.jpg
449 https://www.dogstrust.org.uk/dogimages/1236947_zara_20200207032842_zara-3rd_800.jpg


In [359]:
img_save('dog', 3)

TOTAL ESTIMATED MATCHES: 489
450 https://dogbarkingcontroldevices.com/wp-content/uploads/2020/02/dog_barking_control_devices_safely_and_humanely_control_dog_barking.jpg
451 https://www.amylouisejackson.com/wp-content/uploads/dog-wallpaper-4k-games-archives-page-49-of-180-free-hd-wallpapers-4k-of-dog-wallpaper-4k.jpg
452 https://www.coques-telephone.com/10520-large_default/samsung-galaxy-s6-case-chihuahua-dog.jpg
453 https://www.summitdogrescue.org/uploads/1/7/4/1/17419925/noodle-2.jpg
454 https://www.dogideas.net/wp-content/uploads/main-image-Entlebucher-Sennenhund.jpeg
455 https://png4free.com/wp-content/uploads/2020/02/Big-Dog-Licks(1)-720x635.jpg
456 https://chicagolanddogrescue.org/wp-content/uploads/2020/02/ravenintakepic12020.jpeg
457 https://www.breedyourdog.com/uploads/listing_images/39834/110929/big_listing_image_1580862003_1.jpeg
458 https://galapagoslastminute.co/wp-content/uploads/2019/11/dog-daycare-eugene-elderly-man-with-his-dog-at-grand-in-or-doggie-daycare-eugene-orego

Format the images and save to arrays

In [8]:
x_cat, Y_cat = img_format('cat')

In [9]:
x_dog, Y_dog = img_format('dog')

In [10]:
x_train = np.concatenate((x_cat, x_dog))

In [11]:
x_train = x_train.reshape(len(x_train), 100, 100, 3, 1)

In [12]:
Y_train = np.concatenate((Y_cat, Y_dog))

In [13]:
Y_train = (Y_train == 'cat').astype(int)

Train NN

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [15]:
nn_model = Sequential()

In [16]:
nn_model.add(Dense(16, input_shape=x_train[0].shape, activation='relu'))
nn_model.add(Dense(16,  activation='relu'))
nn_model.add(Flatten())
nn_model.add(Dense(len(np.unique(Y_train)),  activation='softmax'))

In [17]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100, 100, 3, 16)   32        
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 3, 16)   272       
_________________________________________________________________
flatten (Flatten)            (None, 480000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 960002    
Total params: 960,306
Trainable params: 960,306
Non-trainable params: 0
_________________________________________________________________


In [18]:
nn_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

nn_model.fit(x=x_train,y=Y_train, epochs=10)

Train on 749 samples
Epoch 1/10
749/749 [==============================] - 7s 9ms/sample - loss: 495.5365 - accuracy: 0.5247
Epoch 2/10
749/749 [==============================] - 6s 8ms/sample - loss: 83.1602 - accuracy: 0.5340
Epoch 3/10
749/749 [==============================] - 6s 8ms/sample - loss: 28.6493 - accuracy: 0.6582
Epoch 4/10
749/749 [==============================] - 6s 8ms/sample - loss: 16.1774 - accuracy: 0.7049
Epoch 5/10
749/749 [==============================] - 7s 9ms/sample - loss: 31.1314 - accuracy: 0.6502
Epoch 6/10
749/749 [==============================] - 6s 8ms/sample - loss: 23.4742 - accuracy: 0.6462
Epoch 7/10
749/749 [==============================] - 6s 8ms/sample - loss: 11.5750 - accuracy: 0.7383
Epoch 8/10
749/749 [==============================] - 6s 8ms/sample - loss: 14.0945 - accuracy: 0.7049
Epoch 9/10
749/749 [==============================] - 6s 8ms/sample - loss: 6.1774 - accuracy: 0.7530
Epoch 10/10
749/749 [==============================]

In [19]:
nn_model.predict(x_train[6].reshape(1, 100, 100, 3, 1).astype(float))

array([[0., 1.]], dtype=float32)